<a href="https://colab.research.google.com/github/Dipak703/Chat_gen_RNN/blob/main/chatgen_krishna_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.layers import LSTM, Bidirectional
from keras.optimizers import Adam
import json

In [ ]:
token = Tokenizer(oov_token="$_$")

In [ ]:
with open("Lord_Krishna.txt",'r') as da:
    data = da.read()
    text = data.lower().split( "\n")
texts =[]
ptext =[]
for items in text:
    ptext.append(items.split("\t"))
for item in ptext:
    texts.append(item[0])
print(len(text))
print(texts)
print(data)
for items in texts:
    if items == "":
        texts.remove(items)
    if len(items)<4:
        texts.remove(items)



[['lord krishna is the symbol of love and intelligence. lord krishna gave specific knowledge that still helps the world through their entire life written in bhagavad gita. those krishna quotes that krishna gave to arjuna should be must be known by everyone to get a pleasant experience.'], [''], ['he was the eighth incarnations (avatar) of lord vishnu. there are 700 verses in bhagavad gita written in sanskrit and translated in hindi and english.'], [''], ['today, the whole world is recognizing the presence of the holy book and the lesson that krishna gave during mahabharata. from the birth to death, it tells you how to live and how to spend your life, why you have been born and why death is a necessary factor.'], ['"no one who does good work will ever come to a bad end, either here or in the world to come.” ― lord krishna'], ['“reshape yourself through the power of your will; never let yourself be degraded by self-will. the will is the only friend of the self, and the will is the only e

In [ ]:
token.fit_on_texts(texts)
print(token.word_index)


{'$_$': 1, 'the': 2, 'and': 3, 'to': 4, 'of': 5, 'krishna': 6, 'in': 7, 'a': 8, 'lord': 9, 'is': 10, 'he': 11, 'his': 12, 'that': 13, '”': 14, '―': 15, 'their': 16, 'was': 17, 'will': 18, 'from': 19, 'with': 20, 'kamsa': 21, 'love': 22, 'gave': 23, 'vishnu': 24, 'self': 25, 'on': 26, 'krishna’s': 27, 'vasudeva': 28, 'world': 29, 'be': 30, 'by': 31, 'who': 32, 'good': 33, 'not': 34, 'them': 35, 'would': 36, 'through': 37, 'bhagavad': 38, 'gita': 39, 'those': 40, 'arjuna': 41, 'eighth': 42, 'are': 43, 'birth': 44, 'you': 45, 'live': 46, 'born': 47, 'one': 48, 'only': 49, 'all': 50, 'for': 51, 'as': 52, 'wisdom': 53, 'teachings': 54, 'words': 55, 'earth': 56, 'brahma': 57, 'devaki': 58, 'child': 59, 'when': 60, 'prison': 61, 'infant': 62, 'life': 63, 'written': 64, 'quotes': 65, 'there': 66, 'death': 67, 'it': 68, 'how': 69, 'your': 70, 'why': 71, 'been': 72, 'no': 73, 'work': 74, 'come': 75, 'or': 76, 'yourself': 77, 'power': 78, 'never': 79, 'thy': 80, 'but': 81, 'its': 82, 'time': 83, 

In [ ]:
# Serializing json
json_object = json.dumps(token.word_index)

# Writing to sample.json
with open("dailogtoken.json", "w") as outfile:
    outfile.write(json_object)


In [ ]:
twords = len(token.word_index)+1

In [ ]:
input_seq=[]
listofseq=[]
for line in texts:
    token_seq = token.texts_to_sequences([line])[0]
    listofseq.append(token_seq)
    length=len(token_seq)
real_seq = list(filter(None, listofseq))
for seq in real_seq:
    size = len(seq)
    for i in range(1,size):
        input_seq.append(seq[:i+1])

In [ ]:
max_length = max([len(x) for x in real_seq])
print(len(input_seq))
print(max_length)

828
80


In [ ]:
input_seq = np.array(pad_sequences(input_seq,maxlen= max_length,padding ='pre'))
print(input_seq[0])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 9 6]


In [ ]:
train = input_seq[:,:-1]
label = input_seq[:,-1]
y = tf.keras.utils.to_categorical(label,num_classes=twords)

In [ ]:
print(y.shape)

(828, 408)


In [ ]:
# bhai plz is ko barbar run mat ker new model bante hai
model1 = keras.Sequential([keras.layers.Embedding(twords,200,input_length = max_length-1),
                           keras.layers.Bidirectional(LSTM(150)),
                           keras.layers.Dense(twords, activation='softmax') ])
model1.summary()
adam =Adam(learning_rate =0.1)
model1.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 79, 120)           48960     
                                                                 
 bidirectional (Bidirection  (None, 300)               325200    
 al)                                                             
                                                                 
 dense (Dense)               (None, 408)               122808    
                                                                 
Total params: 496968 (1.90 MB)
Trainable params: 496968 (1.90 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model1.fit(train,y,epochs =50,verbose =1)

Epoch 1/50
26/26 [==============================] - 10s 269ms/step - loss: 19.0752 - accuracy: 0.0169
Epoch 2/50
26/26 [==============================] - 6s 248ms/step - loss: 17.2728 - accuracy: 0.0217
Epoch 3/50
26/26 [==============================] - 6s 232ms/step - loss: 14.2059 - accuracy: 0.0580
Epoch 4/50
26/26 [==============================] - 6s 235ms/step - loss: 11.8158 - accuracy: 0.1171
Epoch 5/50
26/26 [==============================] - 6s 232ms/step - loss: 10.2483 - accuracy: 0.1389
Epoch 6/50
26/26 [==============================] - 6s 236ms/step - loss: 9.4864 - accuracy: 0.1993
Epoch 7/50
26/26 [==============================] - 6s 232ms/step - loss: 8.9790 - accuracy: 0.2560
Epoch 8/50
26/26 [==============================] - 6s 232ms/step - loss: 8.3609 - accuracy: 0.2585
Epoch 9/50
26/26 [==============================] - 6s 232ms/step - loss: 8.1344 - accuracy: 0.2862
Epoch 10/50
26/26 [==============================] - 6s 232ms/step - loss: 8.1846 - accuracy: 

In [ ]:
history = model1.fit(train[:10000],y[:10000],epochs =50,verbose =1)


Epoch 1/50
313/313 [==============================] - 16s 35ms/step - loss: 14.8579 - accuracy: 0.0299
Epoch 2/50
313/313 [==============================] - 3s 11ms/step - loss: 14.2544 - accuracy: 0.0565
Epoch 3/50
313/313 [==============================] - 3s 9ms/step - loss: 14.5283 - accuracy: 0.0782
Epoch 4/50
313/313 [==============================] - 3s 9ms/step - loss: 14.4173 - accuracy: 0.0860
Epoch 5/50
313/313 [==============================] - 3s 9ms/step - loss: 14.9498 - accuracy: 0.0941
Epoch 6/50
313/313 [==============================] - 3s 9ms/step - loss: 14.1658 - accuracy: 0.1058
Epoch 7/50
313/313 [==============================] - 3s 8ms/step - loss: 14.8883 - accuracy: 0.1182
Epoch 8/50
313/313 [==============================] - 3s 9ms/step - loss: 15.6973 - accuracy: 0.1190
Epoch 9/50
313/313 [==============================] - 4s 13ms/step - loss: 15.2714 - accuracy: 0.1333
Epoch 10/50
313/313 [==============================] - 3s 10ms/step - loss: 15.5277 - a

In [ ]:
model1.fit(train[10000:20000],y[10000:20000],epochs =50,verbose =1)


Epoch 1/50
313/313 [==============================] - 3s 9ms/step - loss: 11.8890 - accuracy: 0.4356
Epoch 2/50
313/313 [==============================] - 3s 8ms/step - loss: 11.9986 - accuracy: 0.4332
Epoch 3/50
313/313 [==============================] - 3s 9ms/step - loss: 10.2695 - accuracy: 0.4684
Epoch 4/50
313/313 [==============================] - 3s 10ms/step - loss: 9.9588 - accuracy: 0.4740
Epoch 5/50
313/313 [==============================] - 2s 8ms/step - loss: 10.4971 - accuracy: 0.4739
Epoch 6/50
313/313 [==============================] - 2s 8ms/step - loss: 10.9685 - accuracy: 0.4566
Epoch 7/50
313/313 [==============================] - 2s 8ms/step - loss: 10.1876 - accuracy: 0.4826
Epoch 8/50
313/313 [==============================] - 3s 10ms/step - loss: 10.9552 - accuracy: 0.4676
Epoch 9/50
313/313 [==============================] - 3s 9ms/step - loss: 11.8850 - accuracy: 0.4409
Epoch 10/50
313/313 [==============================] - 3s 8ms/step - loss: 12.1321 - accur

In [ ]:
model1.fit(train[20000:30000],y[20000:30000],epochs =50,verbose =1)


Epoch 1/50
313/313 [==============================] - 2s 8ms/step - loss: 90.7344 - accuracy: 0.0661
Epoch 2/50
313/313 [==============================] - 3s 10ms/step - loss: 45.1408 - accuracy: 0.1579
Epoch 3/50
313/313 [==============================] - 2s 8ms/step - loss: 31.0509 - accuracy: 0.2140
Epoch 4/50
313/313 [==============================] - 3s 8ms/step - loss: 23.1978 - accuracy: 0.2773
Epoch 5/50
313/313 [==============================] - 2s 7ms/step - loss: 19.0610 - accuracy: 0.3171
Epoch 6/50
313/313 [==============================] - 2s 8ms/step - loss: 15.9970 - accuracy: 0.3563
Epoch 7/50
313/313 [==============================] - 3s 10ms/step - loss: 13.9339 - accuracy: 0.3920
Epoch 8/50
313/313 [==============================] - 2s 8ms/step - loss: 13.0264 - accuracy: 0.4071
Epoch 9/50
313/313 [==============================] - 2s 7ms/step - loss: 13.3477 - accuracy: 0.3882
Epoch 10/50
313/313 [==============================] - 2s 7ms/step - loss: 12.7123 - accu

In [ ]:
model1.fit(train[30000:],y[30000:],epochs =50,verbose =1)


Epoch 1/50
313/313 [==============================] - 3s 9ms/step - loss: 54.1274 - accuracy: 0.0812
Epoch 2/50
313/313 [==============================] - 2s 8ms/step - loss: 28.5514 - accuracy: 0.1499
Epoch 3/50
313/313 [==============================] - 3s 11ms/step - loss: 20.6425 - accuracy: 0.2137
Epoch 4/50
313/313 [==============================] - 2s 8ms/step - loss: 16.7302 - accuracy: 0.2641
Epoch 5/50
313/313 [==============================] - 2s 8ms/step - loss: 14.6050 - accuracy: 0.3000
Epoch 6/50
313/313 [==============================] - 2s 8ms/step - loss: 13.4315 - accuracy: 0.3342
Epoch 7/50
313/313 [==============================] - 2s 8ms/step - loss: 14.2792 - accuracy: 0.3137
Epoch 8/50
313/313 [==============================] - 3s 10ms/step - loss: 13.7934 - accuracy: 0.3253
Epoch 9/50
313/313 [==============================] - 2s 8ms/step - loss: 12.8080 - accuracy: 0.3449
Epoch 10/50
313/313 [==============================] - 2s 8ms/step - loss: 13.3735 - accu

In [ ]:
model1.save("dialogbot.keras")

In [ ]:
print(type(token.word_index))
print((token.word_index))
def get_key(val):
   for key, value in token.word_index.items():
        if val == value:
            return key

<class 'dict'>
{'$_$': 1, 'the': 2, 'and': 3, 'to': 4, 'of': 5, 'krishna': 6, 'in': 7, 'a': 8, 'lord': 9, 'is': 10, 'he': 11, 'his': 12, 'that': 13, '”': 14, '―': 15, 'their': 16, 'was': 17, 'will': 18, 'from': 19, 'with': 20, 'kamsa': 21, 'love': 22, 'gave': 23, 'vishnu': 24, 'self': 25, 'on': 26, 'krishna’s': 27, 'vasudeva': 28, 'world': 29, 'be': 30, 'by': 31, 'who': 32, 'good': 33, 'not': 34, 'them': 35, 'would': 36, 'through': 37, 'bhagavad': 38, 'gita': 39, 'those': 40, 'arjuna': 41, 'eighth': 42, 'are': 43, 'birth': 44, 'you': 45, 'live': 46, 'born': 47, 'one': 48, 'only': 49, 'all': 50, 'for': 51, 'as': 52, 'wisdom': 53, 'teachings': 54, 'words': 55, 'earth': 56, 'brahma': 57, 'devaki': 58, 'child': 59, 'when': 60, 'prison': 61, 'infant': 62, 'life': 63, 'written': 64, 'quotes': 65, 'there': 66, 'death': 67, 'it': 68, 'how': 69, 'your': 70, 'why': 71, 'been': 72, 'no': 73, 'work': 74, 'come': 75, 'or': 76, 'yourself': 77, 'power': 78, 'never': 79, 'thy': 80, 'but': 81, 'its': 8

In [ ]:
def get_key(val):
   for key, value in token.word_index.items():
        if val == value:
            return key

In [ ]:
seed_text = "lord Krishna"
next_words = 100

for _ in range(next_words):
  token_list = token.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_length-1, padding='pre')
  predicted = model1.predict(token_list, verbose=0)
  # print(max(predicted[0]))
  a = max(predicted[0])
  predicted=predicted.tolist()
  b = predicted[0].index(a)
  # print(b)
  output_word = get_key(b)
  seed_text += " "+ output_word

print(seed_text)
# with open("output2.txt",'w') as f:
#     f.write(seed_text)

lord Krishna is the symbol of love and intelligence lord krishna would to arjuna is a he fled with a bad is a bad end either infant in is mathura where obstacles like snakes and floods from vasudeva's path that still helps the world through he he jailed by he he he jailed in in in in in in gita gita gita gita gita gita gita the infant to self will is the infant and divine hinduism kings ever come soul merely upon sanskrit they found son knowledge can conquer kings kings is through arrived before can spiritual spiritual that spiritual realm


In [ ]:
model1.save("genmodel.keras")

In [ ]:
model1.save_weights('saved_model/genmodel.keras')